In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import json
import warnings
warnings.filterwarnings('ignore')

In [12]:
# Importing data from csv
attendance_csv = "./Resources/2018-2021_Daily_Attendance_NYC.csv"
attendance_df = pd.read_csv(attendance_csv, encoding="ISO-8859-1")
school_name_csv = "./Resources/School_Name.csv"
school_name_df = pd.read_csv(school_name_csv, encoding="ISO-8859-1")
school_name_df.rename(columns={"DBN":"school_dbn", "School Name":"school_name", "School Level":"school_level"}, inplace=True)
school_name_df

,school_dbn,school_name,school_level
0,01M015,P.S. 015 Roberto Clemente,Elementary
1,01M019,P.S. 019 Asher Levy,Elementary
2,01M020,P.S. 020 Anna Silver,Elementary
3,01M034,P.S. 034 Franklin D. Roosevelt,K-8
4,01M063,The STAR Academy - P.S.63,Elementary
...,...,...,...
1624,75X596,P.S. X596,Elementary
1625,75X721,P.S. X721 - Stephen McSweeney School,High school
1626,75X723,P.S. 723,K-12 all grades
1627,75X754,J. M. Rapport School Career Development,High school


In [14]:
# create dataframe for 2018-2019 school year
school_df_2018 = attendance_df[attendance_df['SchoolYear'] == 20182019]
school_df_2018.sort_values(by=["School DBN"], inplace=True)
school_df_2018.rename(columns={"School DBN":"school_dbn", "Enrolled":"enrolled_2018", "Present":"present_2018", "Absent":"absent_2018", "Released":"released_2018"}, inplace=True)
school_df_2018['enrolled_2018'] = school_df_2018['enrolled_2018'].astype('int')
school_df_2018['present_2018'] = school_df_2018['present_2018'].astype('int')
school_df_2018['absent_2018'] = school_df_2018['absent_2018'].astype('int')
school_df_2018['released_2018'] = school_df_2018['released_2018'].astype('int')
school_df_2018.head()

,school_dbn,Date,SchoolYear,enrolled_2018,present_2018,absent_2018,released_2018
18521,01M015,10/03/2018,20182019,171,163,8,0
18002,01M015,10/09/2018,20182019,170,156,14,0
18001,01M015,10/05/2018,20182019,170,162,8,0
19831,01M015,11/13/2018,20182019,174,160,14,0
19830,01M015,11/09/2018,20182019,174,163,11,0


In [16]:
# group 2018 data by school DBN and obtain total attendance numbers
school_df_2018_DBN = school_df_2018.groupby(['school_dbn'], as_index=False).agg({
    'enrolled_2018':'sum', 'present_2018':'sum', 'absent_2018':'sum', 'released_2018':'sum'})
school_df_2018_DBN_df = pd.DataFrame(school_df_2018_DBN)
school_df_2018_DBN_df.head()

,school_dbn,enrolled_2018,present_2018,absent_2018,released_2018
0,01M015,30908,28609,2278,21
1,01M019,44994,40744,3979,271
2,01M020,85674,78870,6804,0
3,01M034,54900,48554,6309,37
4,01M063,40873,37111,3731,31


In [17]:
# create dataframe for 2019-2020 school year
school_df_2019 = attendance_df[attendance_df['SchoolYear'] == 20192020]
school_df_2019.sort_values(by=["School DBN"], inplace=True)
school_df_2019.rename(columns={"School DBN":"school_dbn", "Enrolled":"enrolled_2019", "Present":"present_2019", "Absent":"absent_2019", "Released":"released_2019"}, inplace=True)
school_df_2019['enrolled_2019'] = school_df_2019['enrolled_2019'].astype('int')
school_df_2019['present_2019'] = school_df_2019['present_2019'].astype('int')
school_df_2019['absent_2019'] = school_df_2019['absent_2019'].astype('int')
school_df_2019['released_2019'] = school_df_2019['released_2019'].astype('int')
school_df_2019.head()

,school_dbn,Date,SchoolYear,enrolled_2019,present_2019,absent_2019,released_2019
22671,01M015,10/02/2019,20192020,181,176,5,0
23819,01M015,01/17/2020,20192020,184,165,19,0
23818,01M015,01/23/2020,20192020,184,166,18,0
23817,01M015,01/24/2020,20192020,184,164,20,0
23421,01M015,12/03/2019,20192020,183,173,10,0


In [18]:
# group 2019 data by school DBN and obtain total attendance numbers
school_df_2019_DBN = school_df_2019.groupby(['school_dbn'], as_index=False).agg({
    'enrolled_2019':'sum', 'present_2019':'sum', 'absent_2019':'sum', 'released_2019':'sum'})
school_df_2019_DBN_df = pd.DataFrame(school_df_2019_DBN)
school_df_2019_DBN_df.head()

,school_dbn,enrolled_2019,present_2019,absent_2019,released_2019
0,01M015,21113,19599,1500,14
1,01M019,27491,25248,2242,1
2,01M020,53695,48929,4762,4
3,01M034,33896,29536,4360,0
4,01M063,26659,24134,2494,31


In [19]:
# merge 2018 & 2019 dataframes to compare attendance
school_df_2018_2019 = pd.merge(school_df_2018_DBN_df, school_df_2019_DBN_df, on='school_dbn')
school_df_2018_2019.head()

,school_dbn,enrolled_2018,present_2018,absent_2018,released_2018,enrolled_2019,present_2019,absent_2019,released_2019
0,01M015,30908,28609,2278,21,21113,19599,1500,14
1,01M019,44994,40744,3979,271,27491,25248,2242,1
2,01M020,85674,78870,6804,0,53695,48929,4762,4
3,01M034,54900,48554,6309,37,33896,29536,4360,0
4,01M063,40873,37111,3731,31,26659,24134,2494,31


In [20]:
# merge with school_name_df to properly identify schools
complete_school_df = pd.merge(school_df_2018_2019, school_name_df, on='school_dbn')
complete_school_df.head()

,school_dbn,enrolled_2018,present_2018,absent_2018,released_2018,enrolled_2019,present_2019,absent_2019,released_2019,school_name,school_level
0,01M015,30908,28609,2278,21,21113,19599,1500,14,P.S. 015 Roberto Clemente,Elementary
1,01M019,44994,40744,3979,271,27491,25248,2242,1,P.S. 019 Asher Levy,Elementary
2,01M020,85674,78870,6804,0,53695,48929,4762,4,P.S. 020 Anna Silver,Elementary
3,01M034,54900,48554,6309,37,33896,29536,4360,0,P.S. 034 Franklin D. Roosevelt,K-8
4,01M063,40873,37111,3731,31,26659,24134,2494,31,The STAR Academy - P.S.63,Elementary


# Connect to local database

In [21]:
# setup database connection
db_connection_string = "postgres:Behala1756@localhost:5432/NYC_School_DB"
engine = create_engine(f'postgresql://{db_connection_string}')

In [22]:
# check for tables
engine.table_names()

['schools']

In [30]:
# use pandas to load data to schools table
complete_school_df.to_sql(name='schools', con=engine, if_exists='append', index=False)

# confirming the data was added to the database
pd.read_sql_query('select * from schools', con=engine).head(10)

,school_dbn,enrolled_2018,present_2018,absent_2018,released_2018,enrolled_2019,present_2019,absent_2019,released_2019,school_name,school_level
0,01M015,30908,28609,2278,21,21113,19599,1500,14,P.S. 015 Roberto Clemente,Elementary
1,01M019,44994,40744,3979,271,27491,25248,2242,1,P.S. 019 Asher Levy,Elementary
2,01M020,85674,78870,6804,0,53695,48929,4762,4,P.S. 020 Anna Silver,Elementary
3,01M034,54900,48554,6309,37,33896,29536,4360,0,P.S. 034 Franklin D. Roosevelt,K-8
4,01M063,40873,37111,3731,31,26659,24134,2494,31,The STAR Academy - P.S.63,Elementary
5,01M064,40745,36098,4604,43,27353,24809,2544,0,P.S. 064 Robert Simon,Elementary
6,01M110,64954,60953,3960,41,45066,42329,2737,0,P.S. 110 Florence Nightingale,Elementary
7,01M134,51602,46071,5514,17,28408,25695,2692,21,P.S. 134 Henrietta Szold,Elementary
8,01M140,67989,60175,7814,0,42624,37784,4809,31,P.S. 140 Nathan Straus,K-8
9,01M142,62776,56863,5790,123,37733,34146,3505,82,P.S. 142 Amalia Castro,Elementary


# Performing various queries

In [29]:
# compare enrollment between 2018 & 2019 by school
pd.read_sql_query('select school_name, school_level, enrolled_2018, enrolled_2019 from schools', con=engine).head(10)

,school_name,school_level,enrolled_2018,enrolled_2019
0,P.S. 015 Roberto Clemente,Elementary,30908,21113
1,P.S. 019 Asher Levy,Elementary,44994,27491
2,P.S. 020 Anna Silver,Elementary,85674,53695
3,P.S. 034 Franklin D. Roosevelt,K-8,54900,33896
4,The STAR Academy - P.S.63,Elementary,40873,26659
5,P.S. 064 Robert Simon,Elementary,40745,27353
6,P.S. 110 Florence Nightingale,Elementary,64954,45066
7,P.S. 134 Henrietta Szold,Elementary,51602,28408
8,P.S. 140 Nathan Straus,K-8,67989,42624
9,P.S. 142 Amalia Castro,Elementary,62776,37733


In [31]:
# compare attendance between 2018 & 2019 by school
pd.read_sql_query('select school_name, school_level, present_2018, present_2019 from schools', con=engine).head(10)

,school_name,school_level,present_2018,present_2019
0,P.S. 015 Roberto Clemente,Elementary,28609,19599
1,P.S. 019 Asher Levy,Elementary,40744,25248
2,P.S. 020 Anna Silver,Elementary,78870,48929
3,P.S. 034 Franklin D. Roosevelt,K-8,48554,29536
4,The STAR Academy - P.S.63,Elementary,37111,24134
5,P.S. 064 Robert Simon,Elementary,36098,24809
6,P.S. 110 Florence Nightingale,Elementary,60953,42329
7,P.S. 134 Henrietta Szold,Elementary,46071,25695
8,P.S. 140 Nathan Straus,K-8,60175,37784
9,P.S. 142 Amalia Castro,Elementary,56863,34146


In [32]:
# compare absence between 2018 & 2019 by school
pd.read_sql_query('select school_name, school_level, absent_2018, absent_2019 from schools', con=engine).head(10)

,school_name,school_level,absent_2018,absent_2019
0,P.S. 015 Roberto Clemente,Elementary,2278,1500
1,P.S. 019 Asher Levy,Elementary,3979,2242
2,P.S. 020 Anna Silver,Elementary,6804,4762
3,P.S. 034 Franklin D. Roosevelt,K-8,6309,4360
4,The STAR Academy - P.S.63,Elementary,3731,2494
5,P.S. 064 Robert Simon,Elementary,4604,2544
6,P.S. 110 Florence Nightingale,Elementary,3960,2737
7,P.S. 134 Henrietta Szold,Elementary,5514,2692
8,P.S. 140 Nathan Straus,K-8,7814,4809
9,P.S. 142 Amalia Castro,Elementary,5790,3505


In [33]:
# compare release between 2018 & 2019 by school
pd.read_sql_query('select school_name, school_level, released_2018, released_2019 from schools', con=engine).head(10)

,school_name,school_level,released_2018,released_2019
0,P.S. 015 Roberto Clemente,Elementary,21,14
1,P.S. 019 Asher Levy,Elementary,271,1
2,P.S. 020 Anna Silver,Elementary,0,4
3,P.S. 034 Franklin D. Roosevelt,K-8,37,0
4,The STAR Academy - P.S.63,Elementary,31,31
5,P.S. 064 Robert Simon,Elementary,43,0
6,P.S. 110 Florence Nightingale,Elementary,41,0
7,P.S. 134 Henrietta Szold,Elementary,17,21
8,P.S. 140 Nathan Straus,K-8,0,31
9,P.S. 142 Amalia Castro,Elementary,123,82
